In [1]:
import pandas as pd
import numpy as np

import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import FunctionTransformer

from nltk import tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


from time import time

In [2]:
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('data/ufo.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5
0,0,5/6/2017 05:00,"Camp McGregor, NM",Light,10 minute,Light seen over mountain's east of Camp McGre...,Report appears to us to be consistent with t...
1,1,5/5/2017 11:30,"Austin, TX",Disk,3 second,"Flying saucer descends, possibly lands in Nor...",We would like to communicate with this witne...
2,2,5/5/2017 03:00,"El Mirage, AZ",Circle,30 second,"While letting my dog out, a very bright white...",[]
3,3,5/4/2017 21:27,"Phoenix, AZ",Circle,15 second,Orange round sphere. Orange glowing sphere f...,"We have amended the time above, to reflect a..."
4,4,5/4/2017 18:30,"Phoenix, AZ",Teardrop,5 minute,Flying corkscrews Looking to th east at abou...,Source of the report elects to remain anonym...


In [5]:
text_pre = df.iloc[:, 5]
text_pre

0         Light seen over mountain's east of Camp McGre...
1         Flying saucer descends, possibly lands in Nor...
2         While letting my dog out, a very bright white...
3         Orange round sphere.  Orange glowing sphere f...
4         Flying corkscrews  Looking to th east at abou...
                               ...                        
11926     reddish orange triangular pattern of lights. ...
11927     6 bright red glowing spheres.  Six red sphere...
11928     In daylight, oval object traveled across sky ...
11929     Triangular craft cary nc  On Sturdivant, saw ...
11930     I've been watching videos on Youtube then, I ...
Name: 4, Length: 11931, dtype: object

In [7]:
for idx, doc in enumerate(df.iloc[:, 5]):
    #df.iloc[:, 5].str.replace('\d+', '') # for digits
    df.iloc[idx, 5] = doc.lower()

In [10]:
df.iloc[:, 5] = df.iloc[:, 5].str.replace('\d+', '') # for digits
df.iloc[:, 5] = df.iloc[:, 5].str.replace(r'(\b\w{1,2}\b)', '') # for words
#df.iloc[:, 5] = df.iloc[:, 5].str.replace('[^\w\s]', '') # for punctuation 
df.iloc[:, 5] = df.iloc[:, 5].str.replace(r'[^\w\s]+', '')
df.iloc[:, 5] = df.iloc[:, 5].str.lower()
#df.iloc[:, 5] = df.iloc[:, 5].str.replace(string.punctuation, '') # for punctuation 

In [11]:
text = df.iloc[:, 5]
text

0         light seen over mountain east  camp mcgregor ...
1         flying saucer descends possibly lands  north ...
2         while letting  dog out  very bright white cir...
3         orange round sphere  orange glowing sphere fl...
4         flying corkscrews  looking   east  about   sa...
                               ...                        
11926     reddish orange triangular pattern  lights   a...
11927      bright red glowing spheres  six red spheres ...
11928      daylight oval object traveled across sky fro...
11929     triangular craft cary    sturdivant saw silen...
11930      been watching videos  youtube then  look out...
Name: 4, Length: 11931, dtype: object

 [pipelinize source](https://evisionindia.wordpress.com/2020/03/06/setting-up-text-preprocessing-pipeline-using-scikit-learn-and-spacy-learn-how-to-tokenize-lemmatize-remove-stop-words-and-punctuation-with-sklearn-pipelines/)

In [12]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk import PorterStemmer

In [5]:
# from string import *

# class PorterTokenizer:
#      def __init__(self):
            
#             self.ps = PorterStemmer()
#             self.lower      = lower
#             self.strip      = strip
#             self.stopwords  = stopwords or set(sw.words('english'))
#             self.punct      = punct or set(string.punctuation)
#             self.lemmatizer = WordNetLemmatizer()
#      def __call__(self, doc):
#          return [self.ps.stem(t) for t in word_tokenize(doc)]

In [13]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
# from nltk import word_tokenize          
# from nltk.stem import WordNetLemmatizer 
# class LemmaTokenizer(object):
#     def __init__(self):
#         self.wnl = WordNetLemmatizer()
#     def __call__(self, articles):
#         return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

# tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
#                                 strip_accents = 'unicode', # works 
#                                 stop_words = 'english', # works
#                                 lowercase = True, # works
#                                 max_df = 0.5, # works
#                                 min_df = 10) # works

In [14]:
from sklearn.decomposition import NMF

In [15]:
n_components = 3
n_features=10000

In [16]:
vect = TfidfVectorizer(stop_words='english', max_features=n_features,
                       ngram_range=(1,2),
                       tokenizer=LemmaTokenizer(),
                       strip_accents = 'unicode', # works 
                       lowercase = True, # works
                       max_df = 0.5, # works
                       min_df = 10 # works
                       #, tokenizer=PorterTokenizer()
                      )
words = vect.fit_transform(text)
V = words.toarray()
V.shape

/home/feli/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(11931, 10000)

In [17]:
nmf = NMF(n_components=n_components)
nmf.fit(V)
W = nmf.transform(V)
H = nmf.components_

nmf.reconstruction_err_

105.9345643846546

In [18]:
H.shape

(3, 10000)

In [19]:
feature_names = vect.get_feature_names()


In [20]:
index_val = np.argsort(H)[:, -1:-11:-1]

for i, lat_feat in enumerate(index_val):
    print('%d: %s'%(i+1,', '.join([feature_names[n] for n in lat_feat])))


1: like, bright, saw, craft, just, looked, star, red, moving, white
2: provides, information, anonymous, elect remain, elect, contact information, remain, anonymous provides, remain totally, totally anonymous
3: object, object wa, appeared, shaped, flying, approximately, moving, east, observed, west


In [21]:
print("Performing dimensionality reduction using LSA")
t0 = time()
# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(words)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print()

Performing dimensionality reduction using LSA
done in 0.221477s
Explained variance of the SVD step: 2%



In [22]:
km = KMeans(n_clusters=10, init='k-means++', max_iter=100, n_init=1,
            verbose=False)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(labels, km.labels_))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, km.labels_, sample_size=1000))

# print()

Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=False)
done in 0.128s



In [ ]:
km.

In [23]:
print("Top terms per cluster:")
original_space_centroids = svd.inverse_transform(km.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
#     else:
#         order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vect.get_feature_names()
for i in range(1, n_components +1):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind], end='')
    print()

Top terms per cluster:
Cluster 1: like saw bright craft looked
Cluster 2: provides information anonymous elect remain elect
Cluster 3: object bright like saw moving
